# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

In [83]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [101]:
spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()

df = spark.read.json("data/sparkify_log_small.json")

user_log = spark.read.json("data/sparkify_log_small.json")

In [80]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Question 1

Which page did user id "" (empty string) NOT visit?

In [ ]:
# TODO: write your code to answer question 1
# My answer

user_1 = user_log.select(['userId','page']).where(user_log.userId== "").groupby(user_log.page).count().show()

In [99]:
# provided solution
blank_pages= df.filter(df.userId == "") \
    .select(col('page') \
    .alias('blank_pages')) \
    .dropDuplicates()
blank_pages.show()

+-----------+
|blank_pages|
+-----------+
|       Home|
|      About|
|      Login|
|       Help|
+-----------+



In [91]:
all_pages = df.select('page').dropDuplicates()
all_pages.show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|            Home|
|       Downgrade|
|          Logout|
|   Save Settings|
|           About|
|        Settings|
|           Login|
|        NextSong|
|            Help|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



In [100]:
# find values in all_pages that are not in blank_pages
# these are the pages that the blank user did not go to
for row in set(all_pages.collect()) - set(blank_pages.collect()):
    print(row.page)

Submit Downgrade
Error
Save Settings
Upgrade
Settings
Logout
Submit Upgrade
NextSong
Downgrade


In [37]:
user_log.select("page").dropDuplicates().sort("page").show()

+----------------+
|            page|
+----------------+
|           About|
|       Downgrade|
|           Error|
|            Help|
|            Home|
|           Login|
|          Logout|
|        NextSong|
|   Save Settings|
|        Settings|
|Submit Downgrade|
|  Submit Upgrade|
|         Upgrade|
+----------------+



# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [50]:
# TODO: use this space to explore the behavior of the user with an empty string
# wrong!! not the right code
user_log.select(['userId','level','status','page']).where(user_log.userId =="").dropDuplicates().show()

+------+-----+------+-----+
|userId|level|status| page|
+------+-----+------+-----+
|      | free|   200|About|
|      | free|   200| Home|
|      | free|   200| Help|
|      | free|   307|Login|
|      | paid|   200|About|
|      | paid|   200| Help|
|      | paid|   200| Home|
|      | paid|   307|Login|
+------+-----+------+-----+



# Question 3

How many female users do we have in the data set?

In [62]:
# TODO: write your code to answer question 3
user_female = user_log.select('userid','gender').dropDuplicates().groupby(user_log.gender).count().show()
user_female

+------+-----+
|gender|count|
+------+-----+
|     F|  462|
|  null|    1|
|     M|  501|
+------+-----+



In [ ]:
df.filter(df.gender == 'F') \
    .select('userId', 'gender') \
    .dropDuplicates() \
    .count()

# Question 4

How many songs were played from the most played artist?

In [107]:
# TODO: write your code to answer question 4
user_q4 = user_log.select(['artist','song','userId']) \
                .where(user_log.page =="NextSong") \
                .groupby(user_log.artist) \
                .agg({'Artist':'count'}) \
                .withColumnRenamed('count(Artist)','Artistcount') \
                .sort(desc('Artistcount')) \
                .show()

+--------------------+-----------+
|              artist|Artistcount|
+--------------------+-----------+
|            Coldplay|         83|
|       Kings Of Leon|         69|
|Florence + The Ma...|         52|
|            BjÃÂ¶rk|         46|
|       Dwight Yoakam|         45|
|       Justin Bieber|         43|
|      The Black Keys|         40|
|         OneRepublic|         37|
|                Muse|         36|
|        Jack Johnson|         36|
|           Radiohead|         31|
|        Taylor Swift|         29|
|Barry Tuckwell/Ac...|         28|
|          Lily Allen|         28|
|               Train|         28|
|           Daft Punk|         27|
|           Metallica|         27|
|          Nickelback|         27|
|          Kanye West|         26|
|Red Hot Chili Pep...|         24|
+--------------------+-----------+
only showing top 20 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [108]:
# TODO: write your code to answer question 5

function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())



In [109]:
user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cusum = df.filter((df.page == 'NextSong') | (df.page == 'Home')) \   # bar == bitwise OR
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+



In [111]:
user_window

AttributeError: 'WindowSpec' object has no attribute 'show'